In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')

In [3]:
spotify_qualities_df.head()

,Track,Artist,Album,Year,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity
0,7 rings,Ariana Grande,"thank u, next",2019,178626,4,0.780,0.321,1,-10.747,0,0.3720,0.56200,0.000000,0.0881,0.315,139.961,50
1,Break Free,Ariana Grande,My Everything - Deluxe,2014,214840,4,0.686,0.702,7,-5.325,0,0.0455,0.00637,0.000045,0.2040,0.290,129.948,76
2,Dangerous Woman,Ariana Grande,Dangerous Woman,2016,235946,3,0.664,0.602,4,-5.369,0,0.0412,0.05290,0.000000,0.3560,0.289,134.049,70
3,God is a woman,Ariana Grande,Sweetener,2018,197546,4,0.602,0.658,1,-5.934,1,0.0558,0.02330,0.000060,0.2370,0.268,145.031,75
4,Into You,Ariana Grande,Dangerous Woman,2016,244453,4,0.623,0.734,9,-5.948,1,0.1070,0.01620,0.000002,0.1450,0.370,107.853,71


In [4]:
charts_df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [5]:
lyrics_df.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,0.000000,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,0.000005,0.2950,0.961,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,https://api.spotify.com/v1/audio-analysis/0n2S...,294987.0,4.0
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.590,...,0.807000,0.2990,0.724,115.983,audio_features,5IALWUYK0zDSEmZgb4ICvc,https://api.spotify.com/v1/tracks/5IALWUYK0zDS...,https://api.spotify.com/v1/audio-analysis/5IAL...,244924.0,4.0
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,0.002010,0.0348,0.680,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,https://api.spotify.com/v1/audio-analysis/3XKI...,261973.0,4.0
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


client_id = 'client_id'
client_secret = 'client_secret'

# Encode credentials
auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

# Request access token
auth_response = requests.post(
    'https://accounts.spotify.com/api/token',
    data={'grant_type': 'client_credentials'},
    headers={'Authorization': f'Basic {b64_auth_str}'}
)

# Extract token
access_token = auth_response.json()['access_token']
headers = {'Authorization': f'Bearer {access_token}'}

In [6]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


In [7]:
charts_df['artist'].value_counts()

artist
Taylor Swift                                      1023
Elton John                                         889
Madonna                                            857
Drake                                              787
Kenny Chesney                                      769
                                                  ... 
RiceGum Featuring Alissa Violet                      1
XXXTENTACION Featuring PnB Rock & Trippie Redd       1
Justin Bieber Featuring Burna Boy                    1
Justin Bieber Featuring BEAM                         1
Frankie Vaughan                                      1
Name: count, Length: 10205, dtype: int64

In [8]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              3397 non-null   int64  
 1   song              3397 non-null   object 
 2   band_singer       3397 non-null   object 
 3   songurl           3397 non-null   object 
 4   titletext         3397 non-null   object 
 5   url               3397 non-null   object 
 6   year              3397 non-null   int64  
 7   lyrics            3397 non-null   object 
 8   uri               3397 non-null   object 
 9   danceability      486 non-null    float64
 10  energy            486 non-null    float64
 11  key               486 non-null    float64
 12  loudness          486 non-null    float64
 13  mode              486 non-null    float64
 14  speechiness       486 non-null    float64
 15  acousticness      486 non-null    float64
 16  instrumentalness  486 non-null    float64


In [9]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [10]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  object        
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   datetime        330087 non-null  datetime64[ns]
 8   year            330087 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(3)
memory usage: 21.4+ MB


In [11]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['year', 'song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df

,date,rank_x,song,artist,last-week,peak-rank,weeks-on-board,datetime,year,rank_y,band_singer,songurl,titletext,url,lyrics,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021-11-06,2021,12,The Kid Laroi,/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song),Stay,/wiki/The_Kid_Laroi,I do the same thing I told you that I never wo...,spotify:track:567e29TDzLwZwfDuEpGTwo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021-11-06,2021,12,Justin Bieber,/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song),Stay,/wiki/Justin_Bieber,I do the same thing I told you that I never wo...,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021-11-06,2021,24,Lil Nas X,/wiki/Industry_Baby,Industry Baby,/wiki/Lil_Nas_X,"(D-D-Daytrip took it to ten, hey)\nBaby back, ...",spotify:track:27NovPIUIRrOZoCHxABJwK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021-11-06,2021,24,Jack Harlow,/wiki/Industry_Baby,Industry Baby,/wiki/Jack_Harlow,"(D-D-Daytrip took it to ten, hey)\nBaby back, ...",spotify:track:27NovPIUIRrOZoCHxABJwK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021-11-06,2021,27,Walker Hayes,/wiki/Fancy_Like,Fancy Like,/wiki/Walker_Hayes,"Ayy\nMy girl is bangin', she's so low maintena...",spotify:track:58UKC45GPNTflCN6nwCUeF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55739,2000-01-01,48,Cowboy Take Me Away,Dixie Chicks,52.0,48,6,2000-01-01,2000,95,Dixie Chicks,/wiki/Cowboy_Take_Me_Away,Cowboy Take Me Away,/wiki/Dixie_Chicks,I said I wanna touch the earth\nI wanna break ...,spotify:track:3rXCZRMiMZp0feGcYXpwYX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55740,2000-01-01,50,My Best Friend,Tim McGraw,54.0,50,6,2000-01-01,2000,83,Tim McGraw,/wiki/My_Best_Friend_(Tim_McGraw_song),My Best Friend,/wiki/Tim_McGraw,"I never had no one, that I could count on\nI'v...",spotify:track:5BeY2wY4rGFD1Q3aXO83Bv,0.444,0.484,9.0,-8.311,1.0,0.0289,0.120000,0.000000,0.135,0.263,147.715,audio_features,5BeY2wY4rGFD1Q3aXO83Bv,https://api.spotify.com/v1/tracks/5BeY2wY4rGFD...,https://api.spotify.com/v1/audio-analysis/5BeY...,279867.0,4.0
55741,2000-01-01,56,Say My Name,Destiny's Child,83.0,56,2,2000-01-01,2000,6,Destiny's Child,/wiki/Say_My_Name,Say My Name,/wiki/Destiny%27s_Child,"Darkchild '99\n\nSay my name, say my name\nIf ...",spotify:track:7H6ev70Weq6DdpZyyTmUXk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55742,2000-01-01,61,Higher,Creed,61.0,52,17,2000-01-01,2000,11,Creed,/wiki/Higher_(Creed_song),Higher,/wiki/Creed_(band),When dreaming I'm guided to another world\nTim...,spotify:track:1ZozJfi8u9cO2Ob8KwiwNT,0.459,0.830,2.0,-6.254,1.0,0.0364,0.000051,0.000140,0.206,0.431,155.827,audio_features,1ZozJfi8u9cO2Ob8KwiwNT,https://api.spotify.com/v1/tracks/1ZozJfi8u9cO...,https://api.spotify.com/v1/audio-analysis/1Zoz...,316733.0,4.0


In [12]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
artists

["'N Sync",
 "'N Sync Featuring Nelly",
 '112',
 '112 Featuring Foxy Brown',
 '2 Chainz',
 '2 Chainz Featuring Drake',
 '2 Pistols Featuring T-Pain & Tay Dizm',
 '21 Savage',
 '24kGoldn Featuring iann dior',
 '2Pac',
 '3 Doors Down',
 '3LW',
 '3OH!3',
 '3OH!3 Featuring Ke$ha',
 '5 Seconds Of Summer',
 '50 Cent',
 '50 Cent Featuring Justin Timberlake & Timbaland',
 '50 Cent Featuring Nate Dogg',
 '50 Cent Featuring Olivia',
 '6ix9ine',
 '98 Degrees',
 'A Boogie Wit da Hoodie',
 'A Boogie Wit da Hoodie Featuring 6ix9ine',
 'A Boogie Wit da Hoodie Featuring Kodak Black',
 'A Great Big World & Christina Aguilera',
 'A$AP Ferg Featuring Nicki Minaj',
 'AJR',
 'AWOLNATION',
 'Aaliyah',
 'Ace Hood Featuring Future & Rick Ross',
 'Adele',
 'Aerosmith',
 'Akon',
 "Akon Featuring Colby O'Donis & Kardinal Offishall",
 'Akon Featuring Eminem',
 'Akon Featuring Snoop Dogg',
 'Akon Featuring Styles P.',
 'Alan Jackson',
 'Alan Jackson & Jimmy Buffett',
 'Alanis Morissette',
 'Alessia Cara',
 'Alex C

In [28]:
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

0      The Kid LAROI
1      The Kid LAROI
2          Lil Nas X
3          Lil Nas X
4       Walker Hayes
5         Ed Sheeran
6              Drake
7              Drake
8              Drake
9     Olivia Rodrigo
10          Dua Lipa
11            Wizkid
12            Wizkid
13            Wizkid
14          Doja Cat
15          Doja Cat
16     Glass Animals
17          Maneskin
18          Doja Cat
19          Doja Cat
Name: main_artist, dtype: object

In [14]:
spotify_qualities_df.dtypes

Track                object
Artist               object
Album                object
Year                  int64
Duration              int64
Time_Signature        int64
Danceability        float64
Energy              float64
Key                   int64
Loudness            float64
Mode                  int64
Speechiness         float64
Acousticness        float64
Instrumentalness    float64
Liveness            float64
Valence             float64
Tempo               float64
Popularity            int64
dtype: object

In [15]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['Time_Signature'].value_counts()

Time_Signature
4    562
3     51
5      4
1      3
Name: count, dtype: int64

In [16]:
spotify_qualities_df[spotify_qualities_df['Time_Signature'] == 5.0]

,Track,Artist,Album,Year,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity
243,this is what falling in love feels like,JVKE,this is what ____ feels like (Vol. 1-4),2022,120308,5,0.422,0.440,11,-5.530,1,0.0544,0.617,0.000000,0.0837,0.332,128.934,76
410,Chun-Li,Nicki Minaj,Queen,2018,191600,5,0.686,0.730,6,-5.077,0,0.3740,0.278,0.000048,0.1060,0.474,128.943,66
487,Ginger Me,Rema,Ginger Me,2023,205892,5,0.567,0.577,8,-7.833,0,0.1340,0.134,0.000397,0.1450,0.719,200.008,49
559,I Hate U,SZA,SOS,2021,173906,5,0.533,0.417,7,-8.665,0,0.1480,0.514,0.000000,0.1140,0.376,106.534,70


In [17]:
#pip install billboard.py

In [18]:
url = 'https://www.billboard.com/charts/hot-100/'
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
titles = [tag.get_text(strip=True) for tag in soup.select('li ul li h3')]
artists = [tag.get_text(strip=True) for tag in soup.select('li ul li span.c-label') if not tag.attrs.get('class', [])[-1].startswith('c-label--chart')]

# Clean and zip
hot_100_songs = list(zip(titles[:100], artists[:100]))  # top 100 only

# Preview
for i, (title, artist) in enumerate(hot_100_songs[:5], start=1):
    print(f"{i}. {title} by {artist}")

1. Ordinary by Alex Warren
2. What I Want by 1
3. Just In Case by 1
4. Luther by 17
5. I'm The Problem by 1
